# Install git-lfs and Download the Model

In [ ]:
%%sh

conda install -c conda-forge git-lfs
git lfs install
git lfs fetch --all
git lfs pull
git lfs checkout

# Package the Model

In [ ]:
import os
import tarfile

model_path = "model/"
code_path = "code/"

zipped_model_path = "model.tar.gz"

with tarfile.open(zipped_model_path, "w:gz") as tar:
    tar.add("pytorch_model.bin", os.path.join(model_path, "pytorch_model.bin"))
    tar.add("sagemaker_infer.py", os.path.join(code_path, "infer.py"))
    tar.add("model.py", os.path.join(code_path, "model.py"))
    tar.add("sagemaker_requirements.txt", os.path.join(code_path, "requirements.txt"))

# Deploy the Model

In [ ]:
import time

from sagemaker import get_execution_role, Session
from sagemaker.pytorch import PyTorchModel
from sagemaker.serverless import ServerlessInferenceConfig

algorithm_name = "bert-small-ko-word-boundary-detection"
timestamp = time.strftime("%Y-%m-%d-%H-%M-%S", time.gmtime())
endpoint_name = f"{algorithm_name}-{timestamp}"

model = PyTorchModel(
    name=f"{algorithm_name}-model-{timestamp}",
    entry_point="infer.py",
    model_data=zipped_model_path,
    role=get_execution_role(),
    framework_version="2.0",
    py_version="py310",
)

config = ServerlessInferenceConfig(
    memory_size_in_mb=3072,
    max_concurrency=4,
)

predictor = model.deploy(
    serverless_inference_config=config,
    endpoint_name=endpoint_name
)

# Test the Endpoint

In [ ]:
import json

client = Session().sagemaker_runtime_client
body = ["공장 훗날"]

response = client.invoke_endpoint(
    EndpointName=endpoint_name,
    Body=json.dumps(body, ensure_ascii=False),
    ContentType="application/json",
)

response["Body"].read()